In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator

payload=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
#print(payload)
boom=pd.DataFrame()

class algo:
    start="2015-01-01"
    end="2021-12-01"
    cutoff='2021-01-01'
    interval='1mo'
    thr1=1
    thr2=99
    
    def __init__(self,stock):
        self.stock=stock
        
    # get raw stock data       
    def getdata(self):
        df=(yf.download(self.stock,algo.start,algo.end,interval=algo.interval).dropna())
        df['log_current']=np.log(df.Close/df.Open)
        df['stock']=self.stock
        
        return(df)

    # apply indicators (offset 1)
    def indicators(self):
        df=self.stock
        df['log_past']=DailyLogReturnIndicator(close=df.Close.shift(1)).daily_log_return()
        df=df.dropna()
        
        return(df)
    
    # calculate percentiles based on the hustorical period (cutoff)
    def percentiles(self):
        df=self.stock
        df=(df[df.index<algo.cutoff])
        p1=np.percentile(df.log_past,algo.thr1)
        p2=np.percentile(df.log_past,algo.thr2)
        
        return(p1,p2)
    
    # apply trade conditions
    def trade(self,percs):
        df=self.stock
        df=(df[df.index>=algo.cutoff])
        df['trade_pass']=np.select([(df.log_past<percs[0]),(df.log_past>percs[1])],[1,-1],default=0)
        
        return(df)
    
k=0
l1=0
l2=10

for i in range (l1,l2):
    try:
        rawdata=algo(payload[i]).getdata()
        ta=algo(rawdata).indicators()
        percs=algo(ta).percentiles()
        trades=algo(ta).trade(percs)
        boom=pd.concat([boom,trades])
        
        
        
    except Exception as e:
        print(e)
        k=k+1

#print(boom)
long=boom[boom['trade_pass']==1]
short=boom[boom['trade_pass']==-1]        

group=([boom,long,short])
#print(boom.shape,long.shape,short.shape)
for i in range(0,3): 
    trade_count=group[i].shape[0]
    win_rate=(((group[i][group[i]['log_current']>0]).shape[0]))/trade_count
    avg_per_trade=np.average(group[i].log_current)/trade_count
    minimum=np.min(group[i].log_current)
    print(trade_count,win_rate,avg_per_trade,minimum)
    
short.to_csv('short_trades.csv')
long.to_csv('long_trades.csv')
#trade_count=boom.shape[1]
#win_rate=(((boom[boom['log_current']>0]).shape[1]))/trade_count
#avg_per_trade=np.average(boom.log_current)/trade_count
#minimum=np.min(boom.log_current)
#print(trade_count,win_rate,avg_per_trade,minimum)
#print(np.nan_to_num(temp.reshape(i+1-k-l1,5),nan=0))
#dfout=pd.DataFrame(temp.reshape(i+1-k-l1,5))
#dfout.loc[dfout[3] == 'nan', 3] = 0
#print(dfout)
#dfout.to_csv('2021_bt.csv')
#boom.to_csv('boomer_trades.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
(110, 10) (10, 10) (13, 10)
110 0.5181818181818182 5.900220060117728e-05 -0.29795343061798885
10 0.7 0.003915873539686509 -0.019952537115531896
13 0.5384615384615384 0.0011046911851065448 -0.06899287692564178


In [17]:
print(boom)


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-10-01  176.250000  183.820007  173.779999  178.679993  177.237610   
2021-10-01  118.199997  129.399994  114.400002  128.889999  128.394272   
2021-10-01  108.250000  114.919998  106.860001  114.669998  113.296448   
2021-02-01  246.100006  261.609985  243.970001  250.899994  248.683517   
2021-10-01  323.369995  359.619995  316.010010  358.790009  357.727966   
2021-08-01   84.029999   86.389999   77.809998   82.370003   82.370003   
2021-02-01  462.279999  506.510010  450.119995  459.670013  459.670013   
2021-10-01  577.520020  659.190002  552.140015  650.359985  650.359985   
2021-06-01   25.570000   27.000000   24.629999   26.070000   25.752127   
2021-08-01   23.920000   25.480000   23.450001   23.870001   23.726572   

                 Volume   dlr_day stock        dlr  trade_pass  
Date                                          